# Build a Dashboard Application with Plotly Dash

In this lab, you will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in
real-time.

This dashboard application contains input components such as a dropdown list and a range slider to
interact with a pie chart and a scatter point chart. You will be guided to build this dashboard application via the following tasks:

* TASK 1: Add a Launch Site Drop-down Input Component
* TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown
* TASK 3: Add a Range Slider to Select Payload
* TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot


## Download a skeleton dashboard application and dataset

First, let’s get the SpaceX Launch dataset for this lab:

Run the following `wget` command line in the terminal to download dataset as `spacex_launch_dash.csv`


In [1]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

## TASK 1: Add a Launch Site Drop-down Input Component
We have four different launch sites and we would like to first see which one has the largest success count. Then,
we would like to select one specific site and check its detailed success rate (class=0 vs. class=1).

As such, we will need a dropdown menu to let us select different launch sites.

## TASK 2: Add a callback function to render `success-pie-chart` based on selected site dropdown

The general idea of this callback function is to get the selected launch site from `site-dropdown` and render a pie chart visualizing launch success counts.

Dash callback function is a type of Python function which will be automatically called by Dash whenever receiving an input component updates, such as a click or dropdown selecting event.



## TASK 3: Add a Range Slider to Select Payload
Next, we want to find if variable payload is correlated to mission outcome. From a dashboard point of view, we
want to be able to easily select different payload range and see if we can identify some visual patterns.

Find and complete a commented `dcc.RangeSlider(id='payload-slider',...)` input with the following attribute:

* `id` to be payload-slider
* `min` indicating the slider starting point, we set its value to be 0 (Kg)
* `max` indicating the slider ending point to, we set its value to be 10000 (Kg)
* `step` indicating the slider interval on the slider, we set its value to be 1000 (Kg)
* `value` indicating the current selected range, we could set it to be `min_payload` and `max_payload`


## TASK 4: Add a callback function to render the `success-payload-scatter-chart` scatter plot
Next, we want to plot a scatter plot with the x axis to be the payload and the y axis to be the launch outcome (i.e., `class` column).
As such, we can visually observe how payload may be correlated with mission outcomes for selected site(s).

In addition, we want to color-label the Booster version on each scatter point so that we may
observe mission outcomes with different boosters.

Now, let’s add a call function including the following application logic:

* Input to be `[Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")]`
Note that we have two input components, one to receive selected launch site and another to receive selected payload range
* Output to be `Output(component_id='success-payload-scatter-chart', component_property='figure')`
* A `If-Else` statement to check if ALL sites were selected or just a specific launch site was selected
    * If ALL sites are selected, render a scatter plot to display all values for variable `Payload Mass (kg)` and variable `class`. In addition, the point color needs to be set to the booster version i.e., `color="Booster Version Category"`
    * If a specific launch site is selected, you need to filter the `spacex_df` first, and render a scatter chart to show values `Payload Mass (kg)` and `class` for the selected site, and color-label the point using `Boosster Version Category` likewise.

## Python Code
Consider this skeleton Dash app to be completed in this lab:

In [2]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
spacex_df['Launch Outcome'] = ['Success' if c == 1 else 'Failure' for c in spacex_df['class']]
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout and CSS style
app.layout = html.Div(children = [
    html.H1('SpaceX Launch Records Dashboard',
            style = {'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    
    # TASK 1: Add a dropdown list to enable Launch Site selection
    # The default select value is for ALL sites
    dcc.Dropdown(id = 'site-dropdown',
                options = [
                            {'label': 'All Sites',
                                'value': 'ALL'},
                            {'label': 'CCAFS LC-40',
                                'value': 'CCAFS LC-40'},
                            {'label': 'CCAFS SLC-40',
                                'value': 'CCAFS SLC-40'},
                            {'label': 'KSC LC-39A',
                                'value': 'KSC LC-39A'},
                            {'label': 'VAFB SLC-4E',
                                'value': 'VAFB SLC-4E'}],
                value = 'ALL',
                placeholder = 'Launch Site:',
                searchable = True ),

    # TASK 2: Add a pie chart to show the total successful launches count for all sites
    # If a specific launch site was selected, show the Success vs. Failed counts for the site
    html.Div(dcc.Graph(id = 'success-pie-chart')),
    html.Br(),
    html.P("Payload range (Kg):"),

    # TASK 3: Add a slider to select payload range
    # dcc.RangeSlider(id='payload-slider',...)
    dcc.RangeSlider(id = 'payload-slider',
                    min = 0, max = 10000, step = 1000,
                    marks = {0: '0',
                            1000: '1000',
                            2000: '2000',
                            3000: '3000',
                            4000: '4000',
                            5000: '5000',
                            6000: '6000',
                            7000: '7000',
                            8000: '8000',
                            9000: '9000',
                            10000: '10000'},
                    value = [min_payload, max_payload]),

    # TASK 4: Add a scatter chart to show the correlation between payload and launch success
    html.Div(
        dcc.Graph(id = 'success-payload-scatter-chart')),
    ], style = {'height': '100vh'}
)

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id = 'success-pie-chart', component_property = 'figure'),
              Input(component_id = 'site-dropdown', component_property = 'value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values = 'class',
                     names = 'Launch Site',
                     title = 'SpaceX Launch Site Success Distribution (All Sites)')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        filtered_df = filtered_df.groupby(['Launch Site', 'class']).size().reset_index(name = 'class_count')
        fig = px.pie(filtered_df, values = 'class_count', names = filtered_df['class'].map({1: "Success", 0: "Failure"}),
                     title = f"SpaceX Success Rate of {entered_site} Launch Site")
        fig.update_traces(marker = dict(colors=['red', 'green']))
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id = 'success-payload-scatter-chart', component_property = 'figure'),
              [Input(component_id = 'site-dropdown', component_property = 'value'),
               Input(component_id = 'payload-slider', component_property = 'value')])
def get_scatter_chart(entered_site, payload):
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(
        payload[0], payload[1])]
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x = 'Payload Mass (kg)', y = 'Launch Outcome',
                         color = 'Booster Version Category', title = 'Success vs. Failure for Payload Mass and Launch Sites (All Sites)')
        return fig
    else:
        fig = px.scatter(filtered_df[filtered_df['Launch Site'] == entered_site], x = 'Payload Mass (kg)', y = 'Launch Outcome',
                         color = 'Booster Version Category', title = f"Success vs. Failure for Payload Mass and Launch Site {entered_site}")
        return fig


# Run the app
if __name__ == '__main__':
    app.run_server()


/var/folders/tj/37_nx10d1x522smb0bk9pylh0000gn/T/ipykernel_95180/2280978771.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/var/folders/tj/37_nx10d1x522smb0bk9pylh0000gn/T/ipykernel_95180/2280978771.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


`Note: For opening the Dashboard, run the Python Code above and open the provided link in the browser.`

`Interrupt Execution when done using the dashboard.`

`Example:` 

`"[...]` 
`Dash is running on http://123.4.5.6:8050/` 
`[...]"`

## Questions

After visual analysis using the dashboard, you should be able to obtain some insights to answer the following five questions:

### 1) Which site has the largest successful launches?

`The site that has the largest successful launches is the KSC LC-39A, with 41.7% of total successfull launches.`

### 2) Which site has the highest launch success rate?

`The site that has the highest launch success rate is also the KSC LC-39A, with 76.9% of success rate.`

### 3) Which payload range(s) has the highest launch success rate?

`The highest launch success rate has a payload between 3,000 Kg and 4,000 Kg.`

### 4) Which payload range(s) has the lowest launch success rate?

`The lowest launch success rate has a payload between 6,000 Kg and 8,000 Kg.`

### 5) Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?

`The FT Booster version appears to have the highest launch success rate.`